In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
 
from dotenv import load_dotenv
load_dotenv()

True

In [104]:
search_config = {  
    'service_name': os.environ.get('AZURE_AI_SEARCH_SERVICE_NAME'),  
    'api_key': os.environ.get('AZURE_AI_SEARCH_KEY'),  
    'api_version': '2024-05-01-preview' 
}  

In [54]:

import os
import requests  
import json 

# generic helper function
def create_or_update(search_config, asset_type, definitionfile, overrides): 
   
    # Load the item definition from a file    
    with open(definitionfile, 'r') as f:
        payload = json.load(f)
    if overrides:
        payload.update(overrides)

    # URL for the Azure Search data source  
    url = f'https://{search_config["service_name"]}.search.windows.net/{asset_type}/{payload["name"]}?api-version={search_config["api_version"]}'  
  
    # Headers  
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': search_config["api_key"]  
    }  
  
    payload_json = json.dumps(payload)

    # Send the POST request to create the data source  
    response = requests.put(url, headers=headers, data=payload_json)  
  
    # Check the response  
    if response.status_code in [200, 201, 204]:
        print(f"{asset_type}: {payload['name']} created successfully.")  
    else:  
        print(f"Failed. Status code: {response.status_code}, Response: {response}")
        print(response.text)
  


In [55]:
# create the datasource
overrides = {
    "name": "pdf-samples",
    "credentials": {
      "connectionString": os.environ.get('AZURE_STORAGE_CONNECTION_STRING')
    },
}

create_or_update(search_config,"datasources", definitionfile='../definitions/datasource.json', overrides=overrides )

datasources: pdf-samples created successfully.


In [56]:
# create the index
overrides = {
    "name": "example-index-2",
}

create_or_update(search_config,"indexes", definitionfile='../definitions/index.json', overrides=overrides )

indexes: example-index-2 created successfully.


In [59]:
# create the skillset
# NOTE: please update your openai key in the skillset.json file manually for now 
overrides = {
    "name": "example-skillset",
}

create_or_update(search_config,"skillsets", definitionfile='../definitions/skillset.json', overrides=overrides )

skillsets: example-skillset created successfully.


In [60]:
# create the indexer
overrides = {
    "name": "example-indexer",
}

create_or_update(search_config,"indexers", definitionfile='../definitions/indexer.json', overrides=overrides )

indexers: example-indexer created successfully.


In [150]:
def indexer_operations(search_config, indexer_name, operation):
    '''Performs operations on an indexer
    
    Args:
    search_config (dict): The search service configuration
    indexer_name (str): The name of the indexer
    operation (str): The operation to perform on the indexer (e.g. run, reset, status)
    
    '''

    url = f'https://{search_config["service_name"]}.search.windows.net/indexers/{indexer_name}/search.{operation}?api-version={search_config["api_version"]}'

    headers = {
        'Content-Type': 'application/json',
        'api-key': search_config["api_key"]
    }

    if operation == 'status':
        response = requests.get(url, headers=headers)
    else:
        response = requests.post(url, headers=headers)

    if response.status_code in [200, 202, 201, 204]:
        print(f'Success: {response}')
        if operation == "status":
            print(f"Indexer {indexer_name} {operation}:\n\n--- {response.json()['lastResult']['status']} ---")
            print(f" - Items processed: {response.json()['lastResult']['itemsProcessed'] or 'N/A'}")
            print(f" - Items failed: {response.json()['lastResult']['itemsFailed'] or 'N/A'}")

        print(response.text)
    else:
        print(f"Failed. Status code: {response.status_code}, Response: {response}")
        print(response)
        print(response.text)



In [153]:
# get indexer status
indexer_operations(search_config, "example-indexer", "status")

Success: <Response [200]>
Indexer example-indexer status:

--- success ---
 - Items processed: N/A
 - Items failed: N/A
{"@odata.context":"https://aisearch-uk-basic.search.windows.net/$metadata#Microsoft.Azure.Search.V2024_05_01_Preview.IndexerExecutionInfo","name":"example-indexer","status":"running","lastResult":{"status":"success","statusDetail":null,"errorMessage":null,"startTime":"2024-07-29T18:44:44.626Z","endTime":"2024-07-29T18:44:44.89Z","itemsProcessed":0,"itemsFailed":0,"initialTrackingState":"{\r\n  \"lastFullEnumerationStartTime\": \"2024-07-29T18:42:01.056Z\",\r\n  \"lastAttemptedEnumerationStartTime\": \"2024-07-29T18:44:44.651Z\",\r\n  \"nameHighWaterMark\": null\r\n}","finalTrackingState":"{\"LastFullEnumerationStartTime\":\"2024-07-29T18:44:14.6511448+00:00\",\"LastAttemptedEnumerationStartTime\":\"2024-07-29T18:44:14.6511448+00:00\",\"NameHighWaterMark\":null}","mode":"indexingAllDocs","errors":[],"warnings":[],"metrics":null},"executionHistory":[{"status":"success",

In [143]:
# reset the indexer - this will trigger a re-index of all documents in the data source on the next run
indexer_operations(search_config, "example-indexer", "reset")

Success: <Response [204]>



In [152]:
# run the indexer
indexer_operations(search_config, "example-indexer", "run")

Success: <Response [202]>

